#### Import Libraries and Change Directory

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import xlrd
import statsmodels.formula.api as smf

In [2]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Yearly_Player_Rankings')
os.getcwd()

'C:\\Users\\marcd\\Python_Stuff\\fantasy_football\\Yearly_Player_Rankings'

#### Load Yearly Player Data

In [3]:
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-4:] == 'xlsx']
yearly = pd.DataFrame()
for f in files_csv:
    data = pd.read_excel(f)
    yearly = yearly.append(data, ignore_index=True)
yearly.fillna(0, inplace=True)
yearly

,Rk,Year,Player,Tm,FantPos,Age,G,Pass Cmp,Pass Att,Pass Yds,...,Rec Tgt,Rec,Rec Yds,Rec Y/R,Rec TD,Fmb,FL,TD,2PM,2PP
0,1,2019,Christian McCaffrey,CAR,RB,23,16,0,2,0,...,142,116,1005,8.66,4,1.0,0,19,1.0,0.0
1,2,2019,Lamar Jackson,BAL,QB,22,15,265,401,3127,...,0,0,0,0.00,0,9.0,2,7,0.0,0.0
2,3,2019,Derrick Henry,TEN,RB,25,15,0,0,0,...,24,18,206,11.44,2,5.0,3,18,0.0,0.0
3,4,2019,Aaron Jones,GNB,RB,25,16,0,0,0,...,68,49,474,9.67,3,3.0,2,19,0.0,0.0
4,5,2019,Ezekiel Elliott,DAL,RB,24,16,0,0,0,...,71,54,420,7.78,2,3.0,2,14,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,495,2021,Reggie Gilliam,BUF,RB,24,16,0,0,0,...,5,3,23,7.67,0,0.0,0,0,0.0,0.0
1493,496,2021,Chad Henne,KAN,QB,36,4,11,16,82,...,0,0,0,0.00,0,2.0,0,0,0.0,0.0
1494,497,2021,K.J. Hill,LAC,WR,24,6,0,0,0,...,3,3,30,10.00,0,0.0,0,0,0.0,0.0
1495,498,2021,Kylin Hill,GNB,RB,23,8,0,0,0,...,2,1,5,5.00,0,0.0,0,0,0.0,0.0


#### Scoring Points System Modifier

In [4]:
RR_TD = 6     # Rushing/Receiving TDs
RR_YDS = 0.1  # Rushing/Receiving Yards
REC = 1     # Receptions
FL = -2       # Fumble Loses
P_TD = 4      # Passing TDs
P_YDS = 0.04  # Passing Yards
INT = -2      # Interceptions
X_CONV = 2    # 2-point Conversions

yearly['FPTS'] = RR_TD*(yearly['Rush TD'] + yearly['Rec TD']) + RR_YDS*(yearly['Rush Yds'] + yearly['Rec Yds']) + REC*(yearly['Rec']) + \
FL*(yearly['FL']) + P_TD*(yearly['Pass TD']) + P_YDS*(yearly['Pass Yds']) + INT*(yearly['Pass Int']) + X_CONV*(yearly['2PM'] + yearly['2PP'])

yearly['FPTS/G'] = yearly['FPTS'] / yearly['G']

yearly = yearly.round({'FPTS/G': 2})
yearly

,Rk,Year,Player,Tm,FantPos,Age,G,Pass Cmp,Pass Att,Pass Yds,...,Rec Yds,Rec Y/R,Rec TD,Fmb,FL,TD,2PM,2PP,FPTS,FPTS/G
0,1,2019,Christian McCaffrey,CAR,RB,23,16,0,2,0,...,1005,8.66,4,1.0,0,19,1.0,0.0,471.20,29.45
1,2,2019,Lamar Jackson,BAL,QB,22,15,265,401,3127,...,0,0.00,0,9.0,2,7,0.0,0.0,415.68,27.71
2,3,2019,Derrick Henry,TEN,RB,25,15,0,0,0,...,206,11.44,2,5.0,3,18,0.0,0.0,294.60,19.64
3,4,2019,Aaron Jones,GNB,RB,25,16,0,0,0,...,474,9.67,3,3.0,2,19,0.0,0.0,314.80,19.68
4,5,2019,Ezekiel Elliott,DAL,RB,24,16,0,0,0,...,420,7.78,2,3.0,2,14,0.0,0.0,311.70,19.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,495,2021,Reggie Gilliam,BUF,RB,24,16,0,0,0,...,23,7.67,0,0.0,0,0,0.0,0.0,5.60,0.35
1493,496,2021,Chad Henne,KAN,QB,36,4,11,16,82,...,0,0.00,0,2.0,0,0,0.0,0.0,3.28,0.82
1494,497,2021,K.J. Hill,LAC,WR,24,6,0,0,0,...,30,10.00,0,0.0,0,0,0.0,0.0,6.00,1.00
1495,498,2021,Kylin Hill,GNB,RB,23,8,0,0,0,...,5,5.00,0,0.0,0,0,0.0,0.0,3.90,0.49


#### Save Dataframe as an Excel Sheet

In [5]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football')
writer = pd.ExcelWriter('yearly_relevant.xlsx')
yearly.to_excel(writer,'sheet1')
writer.save()

In [ ]:
def get_QB_data(dir, df, year):
    os.chdir(dir)
    files = os.listdir(path)
    files_csv = [f for f in files if f[-3:] == 'csv']
    df = pd.DataFrame()
    WEEK = 1
    for f in files_csv:
        data = pd.read_csv(f)
        data['WEEK'] = WEEK
        df = df.append(data, ignore_index=True)
        WEEK += 1
    df.fillna(0, inplace=True)
    df = df.drop(columns=['Y/A','SACKS','PCT','G','FPTS','FPTS/G','ROST'])
    df = df.rename(columns={'CMP':'PASS_CMP','ATT':'PASS_ATT','YDS':'PASS_YDS','TD':'PASS_TD','INT':'PASS_INT','ATT.1':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
    df['YEAR'] = year
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[-2:-1] + cols[:11]
    df = df[cols]
    pd.set_option('precision', 0)
    df[['PLAYER','TEAM']] = df['PLAYER'].str.split('(', expand=True)
    df['TEAM'] = df['TEAM'].str.replace(r')','')
    cols = df.columns.tolist()
    cols = cols[:4] + cols[-1:] + cols[4:13]
    df = df[cols]
    df = df[df['PLAYER'].notna()]

In [ ]:
dir = 'C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2019/QB'
df = 'QB_2019'
year = 2019
get_QB_data(dir, df, year)

#### Get 2019 Weekly Data

In [129]:
#QB - 2019
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2019/QB')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
QB_2019 = pd.DataFrame()
WEEK = 1
for f in files_csv:
    data = pd.read_csv(f)
    data['WEEK'] = WEEK
    QB_2019 = QB_2019.append(data, ignore_index=True)
    WEEK += 1
QB_2019.fillna(0, inplace=True)
QB_2019 = QB_2019.drop(columns=['Y/A','SACKS','PCT','G','FPTS','FPTS/G','ROST'])
QB_2019 = QB_2019.rename(columns={'CMP':'PASS_CMP','ATT':'PASS_ATT','YDS':'PASS_YDS','TD':'PASS_TD','INT':'PASS_INT','ATT.1':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
QB_2019['YEAR'] = 2019
pd.set_option('precision', 0)
QB_2019[['PLAYER','TEAM']] = QB_2019['PLAYER'].str.split('(', expand=True)
QB_2019['TEAM'] = QB_2019['TEAM'].str.replace(r')','')
QB_2019['POSITION'] = 'QB'
cols = QB_2019.columns.tolist()
cols = cols[12:13] + cols[11:12] + cols[0:2] + cols[-1:] + cols[-2:-1] + cols[2:11]
QB_2019 = QB_2019[cols]
QB_2019 = QB_2019[QB_2019['PLAYER'].notna()]
QB_2019

C:\Users\marcd\AppData\Local\Temp/ipykernel_14712/2931080283.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  QB_2019['TEAM'] = QB_2019['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,PASS_CMP,PASS_ATT,PASS_YDS,PASS_TD,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2019,1,1,Lamar Jackson,QB,BAL,17,20,324,5,0,3,6,0,0
1,2019,1,2,Dak Prescott,QB,DAL,25,32,405,4,0,4,12,0,0
2,2019,1,3,Deshaun Watson,QB,CLE,20,30,268,3,1,4,40,1,0
3,2019,1,4,Matthew Stafford,QB,LAR,27,45,385,3,0,3,22,0,1
4,2019,1,5,Patrick Mahomes II,QB,KC,25,33,378,3,0,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,2019,17,126,Trevor Siemian,QB,CHI,0,0,0,0,0,0,0,0,0
2302,2019,17,127,Marcus Mariota,QB,ATL,0,0,0,0,0,0,0,0,0
2303,2019,17,128,Brett Hundley,QB,BAL,0,0,0,0,0,0,0,0,0
2304,2019,17,129,Sean Mannion,QB,MIN,0,0,0,0,0,0,0,0,0


In [130]:
#RB - 2019
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2019/RB')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
RB_2019 = pd.DataFrame()
WEEK = 1
for f in files_csv:
    data = pd.read_csv(f)
    data['WEEK'] = WEEK
    RB_2019 = RB_2019.append(data, ignore_index=True)
    WEEK += 1
RB_2019.fillna(0, inplace=True)
RB_2019 = RB_2019.drop(columns=['Y/A','LG','20+','Y/R','G','FPTS','FPTS/G','ROST'])
RB_2019 = RB_2019.rename(columns={'ATT':'RUSH_ATT','YDS':'RUSH_YDS','TD':'RUSH_TD','YDS.1':'REC_YDS','TD.1':'REC_TD','Rank':'RANK','Player':'PLAYER'})
RB_2019['YEAR'] = 2019
pd.set_option('precision', 0)
RB_2019[['PLAYER','TEAM']] = RB_2019['PLAYER'].str.split('(', expand=True)
RB_2019['TEAM'] = RB_2019['TEAM'].str.replace(r')','')
RB_2019['POSITION'] = 'RB'
cols = RB_2019.columns.tolist()
cols = cols[11:12] + cols[10:11] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[2:10]
RB_2019 = RB_2019[cols]
RB_2019 = RB_2019[RB_2019['PLAYER'].notna()]
RB_2019

C:\Users\marcd\AppData\Local\Temp/ipykernel_14712/1247154953.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  RB_2019['TEAM'] = RB_2019['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,RUSH_ATT,RUSH_YDS,RUSH_TD,REC,TGT,REC_YDS,REC_TD,FL
0,2019,1,1,Christian McCaffrey,RB,CAR,19,128,2,10,11,81,0,0
1,2019,1,2,Austin Ekeler,RB,LAC,12,58,1,6,7,96,2,0
2,2019,1,3,Derrick Henry,RB,TEN,19,84,1,1,2,75,1,0
3,2019,1,4,Marlon Mack,RB,HOU,25,174,1,0,0,0,0,0
4,2019,1,5,Dalvin Cook,RB,MIN,21,111,2,2,2,9,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999,2019,17,286,Keith Smith,RB,ATL,0,0,0,0,0,0,0,0
5000,2019,17,287,Roosevelt Nix,RB,FA,0,0,0,0,1,0,0,0
5001,2019,17,288,Brennan Clay,RB,DEN,0,0,0,0,0,0,0,0
5002,2019,17,289,Raheem Mostert,RB,MIA,1,-2,0,0,0,0,0,0


In [131]:
#WR - 2019
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2019/WR')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
WR_2019 = pd.DataFrame()
WEEK = 1
for f in files_csv:
    data = pd.read_csv(f)
    data['WEEK'] = WEEK
    WR_2019 = WR_2019.append(data, ignore_index=True)
    WEEK += 1
WR_2019.fillna(0, inplace=True)
WR_2019 = WR_2019.drop(columns=['LG','20+','Y/R','G','FPTS','FPTS/G','ROST'])
WR_2019 = WR_2019.rename(columns={'YDS':'REC_YDS','TD':'REC_TD','ATT':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
WR_2019['YEAR'] = 2019
pd.set_option('precision', 0)
WR_2019[['PLAYER','TEAM']] = WR_2019['PLAYER'].str.split('(', expand=True)
WR_2019['TEAM'] = WR_2019['TEAM'].str.replace(r')','')
WR_2019['POSITION'] = 'WR'
cols = WR_2019.columns.tolist()
cols = cols[11:12] + cols[10:11] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[2:10]
WR_2019 = WR_2019[cols]
WR_2019 = WR_2019[WR_2019['PLAYER'].notna()]
WR_2019

C:\Users\marcd\AppData\Local\Temp/ipykernel_14712/3255645983.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  WR_2019['TEAM'] = WR_2019['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,REC,TGT,REC_YDS,REC_TD,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2019,1,1,Sammy Watkins,WR,GB,9,11,198,3,0,0,0,0
1,2019,1,2,DeSean Jackson,WR,FA,8,9,154,2,0,0,0,0
2,2019,1,3,John Ross,WR,FA,7,12,158,2,0,0,0,0
3,2019,1,4,Marquise Brown,WR,ARI,4,5,147,2,0,0,0,0
4,2019,1,5,DeAndre Hopkins,WR,ARI,8,13,111,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7953,2019,17,451,Ty Montgomery,WR,NE,0,0,0,0,0,0,0,0
7954,2019,17,452,Damiere Byrd,WR,ATL,0,0,0,0,0,0,0,0
7955,2019,17,453,Jarius Wright,WR,FA,0,4,0,0,1,-7,0,0
7956,2019,17,454,Mecole Hardman,WR,KC,0,0,0,0,1,7,0,1


In [132]:
#TE - 2019
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2019/TE')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
TE_2019 = pd.DataFrame()
WEEK = 1
for f in files_csv:
    data = pd.read_csv(f)
    data['WEEK'] = WEEK
    TE_2019 = TE_2019.append(data, ignore_index=True)
    WEEK += 1
TE_2019.fillna(0, inplace=True)
TE_2019 = TE_2019.drop(columns=['LG','20+','Y/R','G','FPTS','FPTS/G','ROST'])
TE_2019 = TE_2019.rename(columns={'YDS':'REC_YDS','TD':'REC_TD','ATT':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
TE_2019['YEAR'] = 2019
pd.set_option('precision', 0)
TE_2019[['PLAYER','TEAM']] = TE_2019['PLAYER'].str.split('(', expand=True)
TE_2019['TEAM'] = TE_2019['TEAM'].str.replace(r')','')
TE_2019['POSITION'] = 'TE'
cols = TE_2019.columns.tolist()
cols = cols[11:12] + cols[10:11] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[2:10]
TE_2019 = TE_2019[cols]
TE_2019 = TE_2019[TE_2019['PLAYER'].notna()]
TE_2019

C:\Users\marcd\AppData\Local\Temp/ipykernel_14712/1715960820.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  TE_2019['TEAM'] = TE_2019['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,REC,TGT,REC_YDS,REC_TD,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2019,1,1,Evan Engram,TE,JAC,11,14,116,1,0,0,0,0
1,2019,1,2,T.J. Hockenson,TE,DET,6,9,131,1,0,0,0,0
2,2019,1,3,Mark Andrews,TE,BAL,8,8,108,1,0,0,0,0
3,2019,1,4,Delanie Walker,TE,FA,5,6,55,2,0,0,0,0
4,2019,1,5,Vernon Davis,TE,FA,4,7,59,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4666,2019,17,263,Tyler Kroft,TE,SF,0,1,0,0,0,0,0,0
4667,2019,17,264,MyCole Pruitt,TE,FA,0,0,0,0,0,0,0,0
4668,2019,17,265,Chris Manhertz,TE,JAC,0,0,0,0,0,0,0,0
4669,2019,17,266,C.J. Uzomah,TE,NYJ,0,0,0,0,0,0,0,0


#### Merge All 2019 Weekly Data Together

In [133]:
weekly_2019 = QB_2019.append([RB_2019, WR_2019, TE_2019])
weekly_2019.fillna(0, inplace=True)

RR_TD = 6     # Rushing/Receiving TDs
RR_YDS = 0.1  # Rushing/Receiving Yards
REC = 1     # Receptions
FL = -2       # Fumble Loses
P_TD = 4      # Passing TDs
P_YDS = 0.04  # Passing Yards
INT = -2      # Interceptions

weekly_2019['FPTS'] = RR_TD*(weekly_2019['RUSH_TD'] + weekly_2019['REC_TD']) + RR_YDS*(weekly_2019['RUSH_YDS'] + weekly_2019['REC_YDS']) + REC*(weekly_2019['REC']) + \
FL*(weekly_2019['FL']) + P_TD*(weekly_2019['PASS_TD']) + P_YDS*(weekly_2019['PASS_YDS']) + INT*(weekly_2019['PASS_INT'])

weekly_2019 = weekly_2019.round({'FPTS': 2})
weekly_2019

weekly_2019

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,PASS_CMP,PASS_ATT,PASS_YDS,PASS_TD,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,FL,REC,TGT,REC_YDS,REC_TD,FPTS
0,2019,1,1,Lamar Jackson,QB,BAL,17,20,324,5,0,3,6,0,0,0,0,0,0,34
1,2019,1,2,Dak Prescott,QB,DAL,25,32,405,4,0,4,12,0,0,0,0,0,0,33
2,2019,1,3,Deshaun Watson,QB,CLE,20,30,268,3,1,4,40,1,0,0,0,0,0,31
3,2019,1,4,Matthew Stafford,QB,LAR,27,45,385,3,0,3,22,0,1,0,0,0,0,28
4,2019,1,5,Patrick Mahomes II,QB,KC,25,33,378,3,0,1,2,0,0,0,0,0,0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4666,2019,17,263,Tyler Kroft,TE,SF,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4667,2019,17,264,MyCole Pruitt,TE,FA,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4668,2019,17,265,Chris Manhertz,TE,JAC,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4669,2019,17,266,C.J. Uzomah,TE,NYJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Add 2019 Schedule / Opponent column

In [134]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Schedule')
schedule_2019 = pd.read_csv('2019_Schedule.csv')
schedule_2019 = schedule_2019.drop(['W18'], axis=1)
schedule_2019 = schedule_2019.rename(columns={'Team':'TEAM'})
schedule_2019 = schedule_2019.melt(id_vars=['TEAM'], var_name='WEEK')
schedule_2019['WEEK'] = schedule_2019['WEEK'].map(lambda x: x.lstrip('W'))
schedule_2019['value'] = schedule_2019['value'].map(lambda x: x.lstrip('@'))
schedule_2019 = schedule_2019.astype({'WEEK': int})
schedule_2019

,TEAM,WEEK,value
0,ARI,1,DET
1,ATL,1,MIN
2,BAL,1,MIA
3,BUF,1,NYJ
4,CAR,1,LAR
...,...,...,...
539,SEA,17,SF
540,SF,17,SEA
541,TB,17,ATL
542,TEN,17,HOU


In [135]:
weekly_2019 = pd.merge(weekly_2019, schedule_2019, on=['TEAM','WEEK'])
weekly_2019 = weekly_2019.rename(columns={'value':'OPP'})
cols = weekly_2019.columns.tolist()
cols = cols[0:6] + cols[-1:] + cols[6:20]
weekly_2019 = weekly_2019[cols]
weekly_2019

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,OPP,PASS_CMP,PASS_ATT,PASS_YDS,...,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,FL,REC,TGT,REC_YDS,REC_TD,FPTS
0,2019,1,1,Lamar Jackson,QB,BAL,MIA,17,20,324,...,0,3,6,0,0,0,0,0,0,34
1,2019,1,45,Tyler Huntley,QB,BAL,MIA,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2019,1,71,Anthony Brown,QB,BAL,MIA,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2019,1,96,Hunter Cantwell,QB,BAL,MIA,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2019,1,144,Brett Hundley,QB,BAL,MIA,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16910,2019,17,109,Trevon Wesco,TE,NYJ,BUF,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16911,2019,17,124,Jeremy Ruckert,TE,NYJ,BUF,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16912,2019,17,199,Kenny Yeboah,TE,NYJ,BUF,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16913,2019,17,206,James Dearth,TE,NYJ,BUF,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Add Opponent Position Defensive Rankings and Remove Irrelevant Players and Add FPTS/G

In [136]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Defense_Rankings')
defense_2019 = pd.read_csv('Defense_Rankings_2019.csv')
defense_2019 = defense_2019.melt(id_vars=['OPP'], var_name='RANK')
defense_2019 = defense_2019.rename(columns={'RANK':'POSITION','value':'RANK'})

weekly_2019 = pd.merge(weekly_2019, defense_2019, on=['OPP','POSITION'])
weekly_2019 = weekly_2019.rename(columns={'RANK_x':'RANK','RANK_y':'OPP_DEF_POS_RANK'})
cols = weekly_2019.columns.tolist()
cols = cols[0:7] + cols[-1:] + cols[7:21]
weekly_2019 = weekly_2019[cols]

weekly_2019 = weekly_2019.astype({'PASS_CMP':int,'PASS_ATT':int,'PASS_YDS':int,'PASS_TD':int,'PASS_INT':int,'RUSH_ATT':int,'RUSH_YDS':int,'RUSH_TD':int,'FL':int,'REC':int,'TGT':int,'REC_YDS':int,'REC_TD':int,'FPTS':float})

weekly_2019_drop = weekly_2019[(weekly_2019.PASS_CMP == 0) & (weekly_2019.PASS_ATT == 0) & (weekly_2019.PASS_YDS == 0) & (weekly_2019.PASS_TD == 0) & (weekly_2019.PASS_INT == 0) & (weekly_2019.RUSH_ATT == 0) & (weekly_2019.RUSH_YDS == 0) & (weekly_2019.RUSH_TD == 0) & \
(weekly_2019.FL == 0) & (weekly_2019.REC == 0) & (weekly_2019.TGT == 0) & (weekly_2019.REC_YDS == 0) & (weekly_2019.REC_TD == 0) & (weekly_2019.FPTS == 0)].index

weekly_2019.drop(weekly_2019_drop, inplace=True)

weekly_2019['FPTS/G'] = weekly_2019.groupby(['PLAYER'])['FPTS'].transform('mean')
weekly_2019['FPTS/G'] = weekly_2019['FPTS/G'].apply(lambda x: round(x, 2))

weekly_2019['FPTS_DIFF'] = weekly_2019['FPTS'] - weekly_2019['FPTS/G']
weekly_2019['FPTS_DIFF'] = weekly_2019['FPTS_DIFF'].apply(lambda x: round(x, 2))


weekly_2019

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,OPP,OPP_DEF_POS_RANK,PASS_CMP,PASS_ATT,...,RUSH_YDS,RUSH_TD,FL,REC,TGT,REC_YDS,REC_TD,FPTS,FPTS/G,FPTS_DIFF
0,2019,1,1,Lamar Jackson,QB,BAL,MIA,31,17,20,...,6,0,0,0,0,0,0,34,28,6e+00
5,2019,2,25,Brian Hoyer,QB,NE,MIA,31,18,39,...,0,0,0,0,0,0,0,6,5,9e-01
8,2019,3,4,Dak Prescott,QB,DAL,MIA,31,29,46,...,18,0,0,0,0,0,0,32,22,1e+01
17,2019,6,14,Carson Wentz,QB,WAS,MIA,31,33,50,...,9,0,1,0,0,0,0,20,17,3e+00
23,2019,7,17,Josh Allen,QB,BUF,MIA,31,13,25,...,28,1,0,0,0,0,0,16,18,-2e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15367,2019,14,1,Hunter Henry,TE,NE,KC,24,0,0,...,0,0,0,8,9,100,2,30,12,2e+01
15368,2019,14,45,Jonnu Smith,TE,NE,KC,24,0,0,...,0,0,0,1,1,8,0,2,7,-5e+00
15372,2019,15,48,Eric Tomlinson,TE,DEN,KC,24,0,0,...,0,0,0,1,1,1,0,1,1,0e+00
15374,2019,15,117,Andrew Beck,TE,DEN,KC,24,0,0,...,0,0,0,0,1,0,0,0,3,-3e+00


#### Save 2019 Weekly Data

In [137]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football')
writer = pd.ExcelWriter('weekly_2019.xlsx')
weekly_2019.to_excel(writer,'sheet1')
writer.save()

#### Get 2020 Weekly Data

In [138]:
#QB - 2020
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2020/QB')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
QB_2020 = pd.DataFrame()
WEEK = 1
for f in files_csv:
    data = pd.read_csv(f)
    data['WEEK'] = WEEK
    QB_2020 = QB_2020.append(data, ignore_index=True)
    WEEK += 1
QB_2020.fillna(0, inplace=True)
QB_2020 = QB_2020.drop(columns=['Y/A','SACKS','PCT','G','FPTS','FPTS/G','ROST'])
QB_2020 = QB_2020.rename(columns={'CMP':'PASS_CMP','ATT':'PASS_ATT','YDS':'PASS_YDS','TD':'PASS_TD','INT':'PASS_INT','ATT.1':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
QB_2020['YEAR'] = 2020
pd.set_option('precision', 0)
QB_2020[['PLAYER','TEAM']] = QB_2020['PLAYER'].str.split('(', expand=True)
QB_2020['TEAM'] = QB_2020['TEAM'].str.replace(r')','')
QB_2020['POSITION'] = 'QB'
cols = QB_2020.columns.tolist()
cols = cols[12:13] + cols[11:12] + cols[0:2] + cols[-1:] + cols[-2:-1] + cols[2:11]
QB_2020 = QB_2020[cols]
QB_2020 = QB_2020[QB_2020['PLAYER'].notna()]
QB_2020

C:\Users\marcd\AppData\Local\Temp/ipykernel_14712/3046409320.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  QB_2020['TEAM'] = QB_2020['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,PASS_CMP,PASS_ATT,PASS_YDS,PASS_TD,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2020,1,1,Russell Wilson,QB,DEN,31,35,322,4,0,3,29,0,0
1,2020,1,2,Aaron Rodgers,QB,GB,32,44,364,4,0,1,2,0,0
2,2020,1,3,Josh Allen,QB,BUF,33,46,312,2,0,14,57,1,2
3,2020,1,4,Lamar Jackson,QB,BAL,20,25,275,3,0,7,45,0,0
4,2020,1,5,Kyler Murray,QB,ARI,26,40,230,1,1,13,91,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2280,2020,17,128,Marcus Mariota,QB,ATL,0,0,0,0,0,0,0,0,0
2281,2020,17,129,Brett Hundley,QB,BAL,0,0,0,0,0,0,0,0,0
2282,2020,17,130,Taylor Heinicke,QB,WAS,0,0,0,0,0,0,0,0,0
2283,2020,17,131,Tim Boyle,QB,DET,0,0,0,0,0,1,-1,0,0


In [139]:
#RB - 2020
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2020/RB')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
RB_2020 = pd.DataFrame()
WEEK = 1
for f in files_csv:
    data = pd.read_csv(f)
    data['WEEK'] = WEEK
    RB_2020 = RB_2020.append(data, ignore_index=True)
    WEEK += 1
RB_2020.fillna(0, inplace=True)
RB_2020 = RB_2020.drop(columns=['Y/A','LG','20+','Y/R','G','FPTS','FPTS/G','ROST'])
RB_2020 = RB_2020.rename(columns={'ATT':'RUSH_ATT','YDS':'RUSH_YDS','TD':'RUSH_TD','YDS.1':'REC_YDS','TD.1':'REC_TD','Rank':'RANK','Player':'PLAYER'})
RB_2020['YEAR'] = 2020
pd.set_option('precision', 0)
RB_2020[['PLAYER','TEAM']] = RB_2020['PLAYER'].str.split('(', expand=True)
RB_2020['TEAM'] = RB_2020['TEAM'].str.replace(r')','')
RB_2020['POSITION'] = 'RB'
cols = RB_2020.columns.tolist()
cols = cols[11:12] + cols[10:11] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[2:10]
RB_2020 = RB_2020[cols]
RB_2020 = RB_2020[RB_2020['PLAYER'].notna()]
RB_2020

C:\Users\marcd\AppData\Local\Temp/ipykernel_14712/294282576.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  RB_2020['TEAM'] = RB_2020['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,RUSH_ATT,RUSH_YDS,RUSH_TD,REC,TGT,REC_YDS,REC_TD,FL
0,2020,1,1,Josh Jacobs,RB,LV,25,93,3,4,6,46,0,0
1,2020,1,2,Christian McCaffrey,RB,CAR,23,97,2,3,4,38,0,0
2,2020,1,3,Ezekiel Elliott,RB,DAL,22,96,1,3,4,31,1,0
3,2020,1,4,Malcolm Brown,RB,FA,18,79,2,3,4,31,0,0
4,2020,1,5,Nyheim Hines,RB,IND,7,28,1,8,8,45,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4794,2020,17,277,Willie Carter,RB,CHI,0,0,0,0,0,0,0,0
4795,2020,17,278,Ty Montgomery,RB,NE,0,0,0,0,0,0,0,0
4796,2020,17,279,Raheem Mostert,RB,MIA,0,0,0,0,0,0,0,0
4797,2020,17,280,Tevin Coleman,RB,NYJ,0,0,0,0,0,0,0,0


In [140]:
#WR - 2020
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2020/WR')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
WR_2020 = pd.DataFrame()
WEEK = 1
for f in files_csv:
    data = pd.read_csv(f)
    data['WEEK'] = WEEK
    WR_2020 = WR_2020.append(data, ignore_index=True)
    WEEK += 1
WR_2020.fillna(0, inplace=True)
WR_2020 = WR_2020.drop(columns=['LG','20+','Y/R','G','FPTS','FPTS/G','ROST'])
WR_2020 = WR_2020.rename(columns={'YDS':'REC_YDS','TD':'REC_TD','ATT':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
WR_2020['YEAR'] = 2020
pd.set_option('precision', 0)
WR_2020[['PLAYER','TEAM']] = WR_2020['PLAYER'].str.split('(', expand=True)
WR_2020['TEAM'] = WR_2020['TEAM'].str.replace(r')','')
WR_2020['POSITION'] = 'WR'
cols = WR_2020.columns.tolist()
cols = cols[11:12] + cols[10:11] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[2:10]
WR_2020 = WR_2020[cols]
WR_2020 = WR_2020[WR_2020['PLAYER'].notna()]
WR_2020

C:\Users\marcd\AppData\Local\Temp/ipykernel_14712/3798587716.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  WR_2020['TEAM'] = WR_2020['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,REC,TGT,REC_YDS,REC_TD,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2020,1,1,Davante Adams,WR,LV,14,17,156,2,0,0,0,0
1,2020,1,2,Calvin Ridley,WR,ATL,9,12,130,2,1,-1,0,0
2,2020,1,3,Adam Thielen,WR,MIN,6,8,110,2,0,0,0,0
3,2020,1,4,Darius Slayton,WR,NYG,6,9,102,2,0,0,0,0
4,2020,1,5,Robbie Anderson,WR,CAR,6,8,114,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7521,2020,17,434,Ty Montgomery,WR,NE,0,0,0,0,0,0,0,0
7522,2020,17,435,Devin Funchess,WR,DET,0,0,0,0,0,0,0,0
7523,2020,17,436,DeAndre Carter,WR,LAC,0,0,0,0,0,0,0,0
7524,2020,17,437,Devin Duvernay,WR,BAL,0,0,0,0,1,-1,0,0


In [141]:
#TE - 2020
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2020/TE')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
TE_2020 = pd.DataFrame()
WEEK = 1
for f in files_csv:
    data = pd.read_csv(f)
    data['WEEK'] = WEEK
    TE_2020 = TE_2020.append(data, ignore_index=True)
    WEEK += 1
TE_2020.fillna(0, inplace=True)
TE_2020 = TE_2020.drop(columns=['LG','20+','Y/R','G','FPTS','FPTS/G','ROST'])
TE_2020 = TE_2020.rename(columns={'YDS':'REC_YDS','TD':'REC_TD','ATT':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
TE_2020['YEAR'] = 2020
pd.set_option('precision', 0)
TE_2020[['PLAYER','TEAM']] = TE_2020['PLAYER'].str.split('(', expand=True)
TE_2020['TEAM'] = TE_2020['TEAM'].str.replace(r')','')
TE_2020['POSITION'] = 'TE'
cols = TE_2020.columns.tolist()
cols = cols[11:12] + cols[10:11] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[2:10]
TE_2020 = TE_2020[cols]
TE_2020 = TE_2020[TE_2020['PLAYER'].notna()]
TE_2020

C:\Users\marcd\AppData\Local\Temp/ipykernel_14712/2464740450.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  TE_2020['TEAM'] = TE_2020['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,REC,TGT,REC_YDS,REC_TD,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2020,1,1,Mark Andrews,TE,BAL,5,6,58,2,0,0,0,0
1,2020,1,2,Dallas Goedert,TE,PHI,8,9,101,1,0,0,0,0
2,2020,1,3,Noah Fant,TE,SEA,5,6,81,1,0,0,0,0
3,2020,1,4,T.J. Hockenson,TE,DET,5,5,56,1,0,0,0,0
4,2020,1,5,Travis Kelce,TE,KC,6,6,50,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4575,2020,17,264,Devin Funchess,TE,DET,0,0,0,0,0,0,0,0
4576,2020,17,265,MyCole Pruitt,TE,FA,0,0,0,0,0,0,0,0
4577,2020,17,266,Chris Manhertz,TE,JAC,0,0,0,0,0,0,0,0
4578,2020,17,267,C.J. Uzomah,TE,NYJ,0,0,0,0,0,0,0,0


#### Merge All 2020 Weekly Data Together

In [142]:
weekly_2020 = QB_2020.append([RB_2020, WR_2020, TE_2020])
weekly_2020.fillna(0, inplace=True)

RR_TD = 6     # Rushing/Receiving TDs
RR_YDS = 0.1  # Rushing/Receiving Yards
REC = 1     # Receptions
FL = -2       # Fumble Loses
P_TD = 4      # Passing TDs
P_YDS = 0.04  # Passing Yards
INT = -2      # Interceptions

weekly_2020['FPTS'] = RR_TD*(weekly_2020['RUSH_TD'] + weekly_2020['REC_TD']) + RR_YDS*(weekly_2020['RUSH_YDS'] + weekly_2020['REC_YDS']) + REC*(weekly_2020['REC']) + \
FL*(weekly_2020['FL']) + P_TD*(weekly_2020['PASS_TD']) + P_YDS*(weekly_2020['PASS_YDS']) + INT*(weekly_2020['PASS_INT'])

weekly_2020 = weekly_2020.round({'FPTS': 2})
weekly_2020

weekly_2020

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,PASS_CMP,PASS_ATT,PASS_YDS,PASS_TD,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,FL,REC,TGT,REC_YDS,REC_TD,FPTS
0,2020,1,1,Russell Wilson,QB,DEN,31,35,322,4,0,3,29,0,0,0,0,0,0,32
1,2020,1,2,Aaron Rodgers,QB,GB,32,44,364,4,0,1,2,0,0,0,0,0,0,31
2,2020,1,3,Josh Allen,QB,BUF,33,46,312,2,0,14,57,1,2,0,0,0,0,28
3,2020,1,4,Lamar Jackson,QB,BAL,20,25,275,3,0,7,45,0,0,0,0,0,0,28
4,2020,1,5,Kyler Murray,QB,ARI,26,40,230,1,1,13,91,1,0,0,0,0,0,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4575,2020,17,264,Devin Funchess,TE,DET,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4576,2020,17,265,MyCole Pruitt,TE,FA,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4577,2020,17,266,Chris Manhertz,TE,JAC,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4578,2020,17,267,C.J. Uzomah,TE,NYJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Add 2020 Schedule / Opponent column

In [143]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Schedule')
schedule_2020 = pd.read_csv('2020_Schedule.csv')
schedule_2020 = schedule_2020.drop(['W18'], axis=1)
schedule_2020 = schedule_2020.rename(columns={'Team':'TEAM'})
schedule_2020 = schedule_2020.melt(id_vars=['TEAM'], var_name='WEEK')
schedule_2020['WEEK'] = schedule_2020['WEEK'].map(lambda x: x.lstrip('W'))
schedule_2020['value'] = schedule_2020['value'].map(lambda x: x.lstrip('@'))
schedule_2020 = schedule_2020.astype({'WEEK': int})
schedule_2020

,TEAM,WEEK,value
0,ARI,1,SF
1,ATL,1,SEA
2,BAL,1,CLE
3,BUF,1,NYJ
4,CAR,1,LV
...,...,...,...
539,SEA,17,SF
540,SF,17,SEA
541,TB,17,ATL
542,TEN,17,HOU


In [144]:
weekly_2020 = pd.merge(weekly_2020, schedule_2020, on=['TEAM','WEEK'])
weekly_2020 = weekly_2020.rename(columns={'value':'OPP'})
cols = weekly_2020.columns.tolist()
cols = cols[0:6] + cols[-1:] + cols[6:20]
weekly_2020 = weekly_2020[cols]
weekly_2020

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,OPP,PASS_CMP,PASS_ATT,PASS_YDS,...,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,FL,REC,TGT,REC_YDS,REC_TD,FPTS
0,2020,1,1,Russell Wilson,QB,DEN,TEN,31,35,322,...,0,3,29,0,0,0,0,0,0,32
1,2020,1,51,Brett Rypien,QB,DEN,TEN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020,1,103,Josh Johnson,QB,DEN,TEN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020,1,16,Melvin Gordon III,RB,DEN,TEN,0,0,0,...,0,15,78,1,1,3,3,8,0,16
4,2020,1,119,Damarea Crockett,RB,DEN,TEN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16927,2020,17,67,Stephen Sullivan,TE,CAR,NO,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16928,2020,17,173,Jared Scott,TE,CAR,NO,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16929,2020,17,187,Josh Babicz,TE,CAR,NO,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16930,2020,17,202,Tommy Tremble,TE,CAR,NO,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Add Opponent Position Defensive Rankings and Remove Irrelevant Players and Add FPTS/G

In [145]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Defense_Rankings')
defense_2020 = pd.read_csv('Defense_Rankings_2020.csv')
defense_2020 = defense_2020.melt(id_vars=['OPP'], var_name='RANK')
defense_2020 = defense_2020.rename(columns={'RANK':'POSITION','value':'RANK'})

weekly_2020 = pd.merge(weekly_2020, defense_2020, on=['OPP','POSITION'])
weekly_2020 = weekly_2020.rename(columns={'RANK_x':'RANK','RANK_y':'OPP_DEF_POS_RANK'})
cols = weekly_2020.columns.tolist()
cols = cols[0:7] + cols[-1:] + cols[7:21]
weekly_2020 = weekly_2020[cols]

weekly_2020 = weekly_2020.astype({'PASS_CMP':int,'PASS_ATT':int,'PASS_YDS':int,'PASS_TD':int,'PASS_INT':int,'RUSH_ATT':int,'RUSH_YDS':int,'RUSH_TD':int,'FL':int,'REC':int,'TGT':int,'REC_YDS':int,'REC_TD':int,'FPTS':float})

weekly_2020_drop = weekly_2020[(weekly_2020.PASS_CMP == 0) & (weekly_2020.PASS_ATT == 0) & (weekly_2020.PASS_YDS == 0) & (weekly_2020.PASS_TD == 0) & (weekly_2020.PASS_INT == 0) & (weekly_2020.RUSH_ATT == 0) & (weekly_2020.RUSH_YDS == 0) & (weekly_2020.RUSH_TD == 0) & \
(weekly_2020.FL == 0) & (weekly_2020.REC == 0) & (weekly_2020.TGT == 0) & (weekly_2020.REC_YDS == 0) & (weekly_2020.REC_TD == 0) & (weekly_2020.FPTS == 0)].index

weekly_2020.drop(weekly_2020_drop, inplace=True)

weekly_2020['FPTS/G'] = weekly_2020.groupby(['PLAYER'])['FPTS'].transform('mean')
weekly_2020['FPTS/G'] = weekly_2020['FPTS/G'].apply(lambda x: round(x, 2))

weekly_2020['FPTS_DIFF'] = weekly_2020['FPTS'] - weekly_2020['FPTS/G']
weekly_2020['FPTS_DIFF'] = weekly_2020['FPTS_DIFF'].apply(lambda x: round(x, 2))

weekly_2020

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,OPP,OPP_DEF_POS_RANK,PASS_CMP,PASS_ATT,...,RUSH_YDS,RUSH_TD,FL,REC,TGT,REC_YDS,REC_TD,FPTS,FPTS/G,FPTS_DIFF
0,2020,1,1,Russell Wilson,QB,DEN,TEN,28,31,35,...,29,0,0,0,0,0,0,32,23,9
3,2020,3,5,Kirk Cousins,QB,MIN,TEN,28,22,30,...,4,0,1,0,0,0,0,23,19,4
7,2020,5,2,Josh Allen,QB,BUF,TEN,28,32,40,...,11,0,1,0,0,0,0,30,24,6
15,2020,7,25,Mitchell Trubisky,QB,PIT,TEN,28,15,21,...,34,0,0,0,0,0,0,13,15,-2
19,2020,8,10,Brandon Allen,QB,CIN,TEN,28,29,37,...,3,0,0,0,0,0,0,23,15,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15392,2020,11,4,Robert Tonyan,TE,GB,IND,4,0,0,...,0,0,0,5,5,50,1,16,12,4
15393,2020,11,13,Marcedes Lewis,TE,GB,IND,4,0,0,...,0,0,0,1,2,18,1,9,4,5
15399,2020,12,6,Austin Hooper,TE,TEN,IND,4,0,0,...,0,0,0,5,7,34,1,14,9,5
15404,2020,13,45,Pharaoh Brown,TE,HOU,IND,4,0,0,...,0,0,0,1,1,4,0,1,4,-2


#### Save Weekly 2020 Data

In [146]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football')
writer = pd.ExcelWriter('weekly_2020.xlsx')
weekly_2020.to_excel(writer,'sheet1')
writer.save()

#### Get 2021 Weekly Data

In [157]:
#QB - 2021
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2021/QB')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
QB_2021 = pd.DataFrame()
WEEK = 1
for f in files_csv:
    data = pd.read_csv(f)
    data['WEEK'] = WEEK
    QB_2021 = QB_2021.append(data, ignore_index=True)
    WEEK += 1
QB_2021.fillna(0, inplace=True)
QB_2021 = QB_2021.drop(columns=['Y/A','SACKS','PCT','G','FPTS','FPTS/G','ROST'])
QB_2021 = QB_2021.rename(columns={'CMP':'PASS_CMP','ATT':'PASS_ATT','YDS':'PASS_YDS','TD':'PASS_TD','INT':'PASS_INT','ATT.1':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
QB_2021['YEAR'] = 2021
pd.set_option('precision', 0)
QB_2021[['PLAYER','TEAM']] = QB_2021['PLAYER'].str.split('(', expand=True)
QB_2021['TEAM'] = QB_2021['TEAM'].str.replace(r')','')
QB_2021['POSITION'] = 'QB'
cols = QB_2021.columns.tolist()
cols = cols[12:13] + cols[11:12] + cols[0:2] + cols[-1:] + cols[-2:-1] + cols[2:11]
QB_2021 = QB_2021[cols]
QB_2021 = QB_2021[QB_2021['PLAYER'].notna()]
QB_2021

C:\Users\marcd\AppData\Local\Temp/ipykernel_14712/882356467.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  QB_2021['TEAM'] = QB_2021['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,PASS_CMP,PASS_ATT,PASS_YDS,PASS_TD,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2021,1,1,Kyler Murray,QB,ARI,21,32,289,4,1,5,20,1,0
1,2021,1,2,Patrick Mahomes II,QB,KC,27,36,337,3,0,5,18,1,0
2,2021,1,3,Jared Goff,QB,DET,38,57,338,3,1,3,14,0,0
3,2021,1,4,Jameis Winston,QB,NO,14,20,148,5,0,6,37,0,0
4,2021,1,5,Tom Brady,QB,TB,32,50,379,4,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2338,2021,18,124,Brett Hundley,QB,BAL,0,0,0,0,0,0,0,0,0
2339,2021,18,125,Sean Mannion,QB,MIN,0,0,0,0,0,0,0,0,0
2340,2021,18,126,Taylor Heinicke,QB,WAS,0,0,0,0,0,0,0,0,0
2341,2021,18,127,Logan Woodside,QB,TEN,0,0,0,0,0,1,-1,0,0


In [158]:
#RB - 2021
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2021/RB')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
RB_2021 = pd.DataFrame()
WEEK = 1
for f in files_csv:
    data = pd.read_csv(f)
    data['WEEK'] = WEEK
    RB_2021 = RB_2021.append(data, ignore_index=True)
    WEEK += 1
RB_2021.fillna(0, inplace=True)
RB_2021 = RB_2021.drop(columns=['Y/A','LG','20+','Y/R','G','FPTS','FPTS/G','ROST'])
RB_2021 = RB_2021.rename(columns={'ATT':'RUSH_ATT','YDS':'RUSH_YDS','TD':'RUSH_TD','YDS.1':'REC_YDS','TD.1':'REC_TD','Rank':'RANK','Player':'PLAYER'})
RB_2021['YEAR'] = 2021
pd.set_option('precision', 0)
RB_2021[['PLAYER','TEAM']] = RB_2021['PLAYER'].str.split('(', expand=True)
RB_2021['TEAM'] = RB_2021['TEAM'].str.replace(r')','')
RB_2021['POSITION'] = 'RB'
cols = RB_2021.columns.tolist()
cols = cols[11:12] + cols[10:11] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[2:10]
RB_2021 = RB_2021[cols]
RB_2021 = RB_2021[RB_2021['PLAYER'].notna()]
RB_2021

C:\Users\marcd\AppData\Local\Temp/ipykernel_14712/2902351181.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  RB_2021['TEAM'] = RB_2021['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,RUSH_ATT,RUSH_YDS,RUSH_TD,REC,TGT,REC_YDS,REC_TD,FL
0,2021,1,1,Christian McCaffrey,RB,CAR,21,98,0,9,9,89,0,0
1,2021,1,2,Joe Mixon,RB,CIN,29,127,1,4,4,23,0,0
2,2021,1,3,Nick Chubb,RB,CLE,15,83,2,2,2,18,0,1
3,2021,1,4,Jamaal Williams,RB,DET,9,54,1,8,9,56,0,0
4,2021,1,5,D'Andre Swift,RB,DET,11,39,0,8,11,65,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4756,2021,18,260,Michael Burton,RB,KC,0,0,0,0,0,0,0,0
4757,2021,18,261,Keith Smith,RB,ATL,0,0,0,0,0,0,0,0
4758,2021,18,262,Duke Johnson Jr.,RB,BUF,0,0,0,0,0,0,0,0
4759,2021,18,263,Tevin Coleman,RB,NYJ,0,0,0,0,0,0,0,0


In [159]:
#WR - 2021
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2021/WR')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
WR_2021 = pd.DataFrame()
WEEK = 1
for f in files_csv:
    data = pd.read_csv(f)
    data['WEEK'] = WEEK
    WR_2021 = WR_2021.append(data, ignore_index=True)
    WEEK += 1
WR_2021.fillna(0, inplace=True)
WR_2021 = WR_2021.drop(columns=['LG','20+','Y/R','G','FPTS','FPTS/G','ROST'])
WR_2021 = WR_2021.rename(columns={'YDS':'REC_YDS','TD':'REC_TD','ATT':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
WR_2021['YEAR'] = 2021
pd.set_option('precision', 0)
WR_2021[['PLAYER','TEAM']] = WR_2021['PLAYER'].str.split('(', expand=True)
WR_2021['TEAM'] = WR_2021['TEAM'].str.replace(r')','')
WR_2021['POSITION'] = 'WR'
cols = WR_2021.columns.tolist()
cols = cols[11:12] + cols[10:11] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[2:10]
WR_2021 = WR_2021[cols]
WR_2021 = WR_2021[WR_2021['PLAYER'].notna()]
WR_2021

C:\Users\marcd\AppData\Local\Temp/ipykernel_14712/3368746448.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  WR_2021['TEAM'] = WR_2021['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,REC,TGT,REC_YDS,REC_TD,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2021,1,1,Amari Cooper,WR,CLE,13,17,139,2,0,0,0,0
1,2021,1,2,Tyreek Hill,WR,MIA,11,15,197,1,1,4,0,0
2,2021,1,3,Deebo Samuel,WR,SF,9,12,189,1,0,0,0,1
3,2021,1,4,Adam Thielen,WR,MIN,9,10,92,2,0,0,0,0
4,2021,1,5,Corey Davis,WR,NYJ,5,7,97,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7780,2021,18,423,Jalen Reagor,WR,PHI,1,1,-6,0,0,0,0,0
7781,2021,18,424,Ray-Ray McCloud,WR,SF,1,1,12,0,0,0,0,1
7782,2021,18,425,Marcus Johnson,WR,SF,0,0,0,0,1,-7,0,0
7783,2021,18,426,Jakeem Grant Sr.,WR,CLE,0,0,0,0,0,0,0,1


In [160]:
#TE - 2021
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Weekly_Player_Rankings/2021/TE')
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[-3:] == 'csv']
TE_2021 = pd.DataFrame()
WEEK = 1
for f in files_csv:
    data = pd.read_csv(f)
    data['WEEK'] = WEEK
    TE_2021 = TE_2021.append(data, ignore_index=True)
    WEEK += 1
TE_2021.fillna(0, inplace=True)
TE_2021 = TE_2021.drop(columns=['LG','20+','Y/R','G','FPTS','FPTS/G','ROST'])
TE_2021 = TE_2021.rename(columns={'YDS':'REC_YDS','TD':'REC_TD','ATT':'RUSH_ATT','YDS.1':'RUSH_YDS','TD.1':'RUSH_TD','Rank':'RANK','Player':'PLAYER'})
TE_2021['YEAR'] = 2021
pd.set_option('precision', 0)
TE_2021[['PLAYER','TEAM']] = TE_2021['PLAYER'].str.split('(', expand=True)
TE_2021['TEAM'] = TE_2021['TEAM'].str.replace(r')','')
TE_2021['POSITION'] = 'TE'
cols = TE_2021.columns.tolist()
cols = cols[11:12] + cols[10:11] + cols[:2] + cols[-1:] + cols[-2:-1] + cols[2:10]
TE_2021 = TE_2021[cols]
TE_2021 = TE_2021[TE_2021['PLAYER'].notna()]
TE_2021

C:\Users\marcd\AppData\Local\Temp/ipykernel_14712/2196072678.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  TE_2021['TEAM'] = TE_2021['TEAM'].str.replace(r')','')


,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,REC,TGT,REC_YDS,REC_TD,RUSH_ATT,RUSH_YDS,RUSH_TD,FL
0,2021,1,1,Rob Gronkowski,TE,FA,8,8,90,2,0,0,0,0
1,2021,1,2,Travis Kelce,TE,KC,6,7,76,2,0,0,0,0
2,2021,1,3,T.J. Hockenson,TE,DET,8,11,97,1,0,0,0,0
3,2021,1,4,Darren Waller,TE,LV,10,19,105,1,0,0,0,0
4,2021,1,5,Juwan Johnson,TE,NO,3,3,21,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4458,2021,18,240,Maxx Williams,TE,ARI,0,0,0,0,0,0,0,0
4459,2021,18,241,Nick Boyle,TE,BAL,0,0,0,0,0,0,0,0
4460,2021,18,242,MyCole Pruitt,TE,FA,0,0,0,0,0,0,0,0
4461,2021,18,243,Blake Bell,TE,KC,0,1,0,0,0,0,0,0


#### Merge All 2020 Weekly Data Together

In [161]:
weekly_2021 = QB_2021.append([RB_2021, WR_2021, TE_2021])
weekly_2021.fillna(0, inplace=True)

RR_TD = 6     # Rushing/Receiving TDs
RR_YDS = 0.1  # Rushing/Receiving Yards
REC = 1     # Receptions
FL = -2       # Fumble Loses
P_TD = 4      # Passing TDs
P_YDS = 0.04  # Passing Yards
INT = -2      # Interceptions

weekly_2021['FPTS'] = RR_TD*(weekly_2021['RUSH_TD'] + weekly_2021['REC_TD']) + RR_YDS*(weekly_2021['RUSH_YDS'] + weekly_2021['REC_YDS']) + REC*(weekly_2021['REC']) + \
FL*(weekly_2021['FL']) + P_TD*(weekly_2021['PASS_TD']) + P_YDS*(weekly_2021['PASS_YDS']) + INT*(weekly_2021['PASS_INT'])

weekly_2021 = weekly_2021.round({'FPTS': 2})
weekly_2021

weekly_2021

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,PASS_CMP,PASS_ATT,PASS_YDS,PASS_TD,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,FL,REC,TGT,REC_YDS,REC_TD,FPTS
0,2021,1,1,Kyler Murray,QB,ARI,21,32,289,4,1,5,20,1,0,0,0,0,0,3e+01
1,2021,1,2,Patrick Mahomes II,QB,KC,27,36,337,3,0,5,18,1,0,0,0,0,0,3e+01
2,2021,1,3,Jared Goff,QB,DET,38,57,338,3,1,3,14,0,0,0,0,0,0,2e+01
3,2021,1,4,Jameis Winston,QB,NO,14,20,148,5,0,6,37,0,0,0,0,0,0,3e+01
4,2021,1,5,Tom Brady,QB,TB,32,50,379,4,2,0,0,0,0,0,0,0,0,3e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4458,2021,18,240,Maxx Williams,TE,ARI,0,0,0,0,0,0,0,0,0,0,0,0,0,0e+00
4459,2021,18,241,Nick Boyle,TE,BAL,0,0,0,0,0,0,0,0,0,0,0,0,0,0e+00
4460,2021,18,242,MyCole Pruitt,TE,FA,0,0,0,0,0,0,0,0,0,0,0,0,0,0e+00
4461,2021,18,243,Blake Bell,TE,KC,0,0,0,0,0,0,0,0,0,0,1,0,0,0e+00


#### Add 2019 Schedule / Opponent column

In [162]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Schedule')
schedule_2021 = pd.read_csv('2021_Schedule.csv')
schedule_2021 = schedule_2021.drop(['W18'], axis=1)
schedule_2021 = schedule_2021.rename(columns={'Team':'TEAM'})
schedule_2021 = schedule_2021.melt(id_vars=['TEAM'], var_name='WEEK')
schedule_2021['WEEK'] = schedule_2021['WEEK'].map(lambda x: x.lstrip('W'))
schedule_2021['value'] = schedule_2021['value'].map(lambda x: x.lstrip('@'))
schedule_2021 = schedule_2021.astype({'WEEK': int})
schedule_2021

,TEAM,WEEK,value
0,ARI,1,TEN
1,ATL,1,PHI
2,BAL,1,LV
3,BUF,1,PIT
4,CAR,1,NYJ
...,...,...,...
539,SEA,17,DET
540,SF,17,HOU
541,TB,17,NYJ
542,TEN,17,MIA


In [163]:
weekly_2021 = pd.merge(weekly_2021, schedule_2021, on=['TEAM','WEEK'])
weekly_2021 = weekly_2021.rename(columns={'value':'OPP'})
cols = weekly_2021.columns.tolist()
cols = cols[0:6] + cols[-1:] + cols[6:20]
weekly_2021 = weekly_2021[cols]
weekly_2021

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,OPP,PASS_CMP,PASS_ATT,PASS_YDS,...,PASS_INT,RUSH_ATT,RUSH_YDS,RUSH_TD,FL,REC,TGT,REC_YDS,REC_TD,FPTS
0,2021,1,1,Kyler Murray,QB,ARI,TEN,21,32,289,...,1,5,20,1,0,0,0,0,0,34
1,2021,1,49,Trace McSorley,QB,ARI,TEN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021,1,55,Jarrett Guarantano,QB,ARI,TEN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021,1,99,Colt McCoy,QB,ARI,TEN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021,1,44,James Conner,RB,ARI,TEN,0,0,0,...,0,16,53,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16927,2021,17,24,Harrison Bryant,TE,CLE,PIT,0,0,0,...,0,0,0,0,0,2,2,33,0,5
16928,2021,17,63,Nakia Griffin-Stewart,TE,CLE,PIT,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16929,2021,17,158,Zaire Mitchell-Paden,TE,CLE,PIT,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16930,2021,17,180,Marcus Santos-Silva,TE,CLE,PIT,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Add Opponent Position Defensive Rankings and Remove Irrelevant Players and Add FPTS/G

In [164]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football/Defense_Rankings')
defense_2021 = pd.read_csv('Defense_Rankings_2021.csv')
defense_2021 = defense_2021.melt(id_vars=['OPP'], var_name='RANK')
defense_2021 = defense_2021.rename(columns={'RANK':'POSITION','value':'RANK'})

weekly_2021 = pd.merge(weekly_2021, defense_2021, on=['OPP','POSITION'])
weekly_2021 = weekly_2021.rename(columns={'RANK_x':'RANK','RANK_y':'OPP_DEF_POS_RANK'})
cols = weekly_2021.columns.tolist()
cols = cols[0:7] + cols[-1:] + cols[7:21]
weekly_2021 = weekly_2021[cols]

weekly_2021 = weekly_2021.astype({'PASS_CMP':int,'PASS_ATT':int,'PASS_YDS':int,'PASS_TD':int,'PASS_INT':int,'RUSH_ATT':int,'RUSH_YDS':int,'RUSH_TD':int,'FL':int,'REC':int,'TGT':int,'REC_YDS':int,'REC_TD':int,'FPTS':float})

weekly_2021_drop = weekly_2021[(weekly_2021.PASS_CMP == 0) & (weekly_2021.PASS_ATT == 0) & (weekly_2021.PASS_YDS == 0) & (weekly_2021.PASS_TD == 0) & (weekly_2021.PASS_INT == 0) & (weekly_2021.RUSH_ATT == 0) & (weekly_2021.RUSH_YDS == 0) & (weekly_2021.RUSH_TD == 0) & \
(weekly_2021.FL == 0) & (weekly_2021.REC == 0) & (weekly_2021.TGT == 0) & (weekly_2021.REC_YDS == 0) & (weekly_2021.REC_TD == 0) & (weekly_2021.FPTS == 0)].index

weekly_2021.drop(weekly_2021_drop, inplace=True)

weekly_2021['FPTS/G'] = weekly_2021.groupby(['PLAYER'])['FPTS'].transform('mean')
weekly_2021['FPTS/G'] = weekly_2021['FPTS/G'].apply(lambda x: round(x, 2))

weekly_2021['FPTS_DIFF'] = weekly_2021['FPTS'] - weekly_2021['FPTS/G']
weekly_2021['FPTS_DIFF'] = weekly_2021['FPTS_DIFF'].apply(lambda x: round(x, 2))

weekly_2021

,YEAR,WEEK,RANK,PLAYER,POSITION,TEAM,OPP,OPP_DEF_POS_RANK,PASS_CMP,PASS_ATT,...,RUSH_YDS,RUSH_TD,FL,REC,TGT,REC_YDS,REC_TD,FPTS,FPTS/G,FPTS_DIFF
0,2021,1,1,Kyler Murray,QB,ARI,TEN,20,21,32,...,20,1,0,0,0,0,0,3e+01,2e+01,1e+01
7,2021,3,29,Matt Ryan,QB,IND,TEN,20,19,28,...,0,0,0,0,0,0,0,2e+00,1e+01,-9e+00
8,2021,3,32,Sam Ehlinger,QB,IND,TEN,20,0,0,...,9,0,0,0,0,0,0,9e-01,4e-01,5e-01
11,2021,4,25,Zach Wilson,QB,NYJ,TEN,20,14,24,...,3,1,0,0,0,0,0,1e+01,1e+01,-2e+00
15,2021,6,1,Josh Allen,QB,BUF,TEN,20,36,54,...,109,1,0,0,0,0,0,4e+01,2e+01,1e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15399,2021,14,31,Noah Fant,TE,SEA,HOU,24,0,0,...,0,0,0,3,4,20,0,5e+00,9e+00,-4e+00
15400,2021,14,60,Colby Parkinson,TE,SEA,HOU,24,0,0,...,0,0,0,1,2,1,0,1e+00,2e+00,-9e-01
15404,2021,16,27,Gerald Everett,TE,LAC,HOU,24,0,0,...,12,0,0,3,3,11,0,5e+00,8e+00,-2e+00
15410,2021,17,9,Tyler Kroft,TE,SF,HOU,24,0,0,...,0,0,0,2,2,20,1,1e+01,5e+00,5e+00


#### Save Weekly 2021 Data

In [165]:
os.chdir('C:/Users/marcd/Python_Stuff/fantasy_football')
writer = pd.ExcelWriter('weekly_2021.xlsx')
weekly_2021.to_excel(writer,'sheet1')
writer.save()